In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
#请在此区域输入代码，代码编辑完成后点击运行，结果将会显示在该单元下方，一个问题一个单元。
#读取新闻训练集和测试集
with open('sohu_train.txt', 'r',encoding="utf-8") as f:
    sohu_train = f.readlines()
with open('sohu_test.txt', 'r',encoding="utf-8") as f:
    sohu_test = f.readlines()
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for new in sohu_train[1:]:
    x_train.append(new.split(',')[2])
    y_train.append(new.split(',')[1])
for new in sohu_test[1:]:
    x_test.append(new.split('\t')[1])
    y_test.append(new.split('\t')[0])

In [22]:
# 将文本转换为数值特征
vectorizer = TfidfVectorizer()
x_train_transformed = vectorizer.fit_transform(x_train)
print(x_train_transformed)
x_test_transformed = vectorizer.transform(x_test)

  (0, 2557192)	0.030832755361714155
  (0, 2154220)	0.007779918352759214
  (0, 367191)	0.05601108682783772
  (0, 1040700)	0.05190596812428182
  (0, 2119869)	0.05601108682783772
  (0, 1480785)	0.037980102423634164
  (0, 1223269)	0.05601108682783772
  (0, 1543825)	0.05601108682783772
  (0, 1846080)	0.05601108682783772
  (0, 945134)	0.05601108682783772
  (0, 1520571)	0.05601108682783772
  (0, 1964257)	0.05601108682783772
  (0, 2061738)	0.05601108682783772
  (0, 1966765)	0.05601108682783772
  (0, 1297846)	0.05601108682783772
  (0, 780897)	0.05601108682783772
  (0, 453985)	0.05601108682783772
  (0, 1569273)	0.05601108682783772
  (0, 1817478)	0.05601108682783772
  (0, 942650)	0.05601108682783772
  (0, 1969453)	0.05601108682783772
  (0, 2335907)	0.0549312993687707
  (0, 1349022)	0.05601108682783772
  (0, 1777426)	0.05601108682783772
  (0, 1847027)	0.05601108682783772
  :	:
  (23548, 254656)	0.08282409669192076
  (23548, 167706)	0.08282409669192076
  (23548, 2395177)	0.16203510001406926
  (2354

In [4]:
# 创建并训练模型
model = MultinomialNB()
model.fit(x_train_transformed, y_train)

# 使用测试数据进行预测
y_pred = model.predict(x_test_transformed)

# 打印分类报告
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          体育       0.97      0.94      0.96      1000
          健康       0.91      0.83      0.87      1000
          女人       0.91      0.91      0.91      1000
          娱乐       0.83      0.87      0.85       999
         房地产       0.81      0.76      0.78      1000
          教育       0.88      0.89      0.89      1000
          文化       0.90      0.75      0.82      1000
          新闻       0.65      0.80      0.72      1000
          旅游       0.96      0.83      0.89      1000
          汽车       0.94      0.88      0.91      1000
          科技       0.88      0.81      0.84      1000
          财经       0.62      0.82      0.71      1000

    accuracy                           0.84     11999
   macro avg       0.85      0.84      0.84     11999
weighted avg       0.85      0.84      0.84     11999



In [23]:
class NaiveBayesClassifier:
    def __init__(self, alpha=1.0):
        self.alpha = alpha
        self.class_log_prior_ = None
        self.feature_log_prob_ = None
        self.classes_ = None

    def fit(self, X, y):
        separated = [X[y == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(i.shape[0] / X.shape[0]) for i in separated]
        self.feature_log_prob_ = []
        self.classes_ = np.unique(y)
        for i in separated:
            count = np.sum(i, axis=0) + self.alpha
            feature_prob = count / (np.sum(count) + self.alpha * X.shape[1])
            self.feature_log_prob_.append(np.log(feature_prob))
        self.feature_log_prob_ = np.array(self.feature_log_prob_).T
        return self

    def predict_log_proba(self, X):
        return np.array([X.multiply(x).sum(axis=1) + c for x, c in zip(self.feature_log_prob_.T, self.class_log_prior_)])

    def predict(self, X):
        log_probs = self.predict_log_proba(X)
        return self.classes_[np.argmax(log_probs, axis=0)]

In [24]:
# 创建并训练模型
model = NaiveBayesClassifier()
model.fit(x_train_transformed, y_train)
y1=model.predict(x_train_transformed)
print(classification_report(y_train, y1))

# 使用测试数据进行预测
y_pred = model.predict(x_test_transformed)

# 打印分类报告
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          体育       0.08      1.00      0.15      1978
          健康       0.00      0.00      0.00      2000
          女人       0.00      0.00      0.00      1997
          娱乐       0.00      0.00      0.00      1934
         房地产       0.00      0.00      0.00      1872
          教育       0.00      0.00      0.00      1979
          文化       0.00      0.00      0.00      1995
          新闻       0.00      0.00      0.00      1935
          旅游       0.00      0.00      0.00      1998
          汽车       0.00      0.00      0.00      1996
          科技       0.00      0.00      0.00      1988
          财经       0.00      0.00      0.00      1877

    accuracy                           0.08     23549
   macro avg       0.01      0.08      0.01     23549
weighted avg       0.01      0.08      0.01     23549

              precision    recall  f1-score   support

          体育       0.08      1.00      0.15      1000
          健康       0.00 